In [61]:
import lxml.html as lh
import pandas as pd
import html2text
import re
import ipywidgets as widgets
from wavelength_color import wavelength_to_rgb

In [62]:
file = open("PrincipleLanthanideTransitions.txt","r", encoding="utf-8")
doc = file.readlines()

In [63]:
headers = []
h = html2text.HTML2Text()
# Ignore converting links from HTML
h.ignore_links = True
for line in doc[11:19]:
    actual_text = h.handle(line)
    actual_text = actual_text.strip()
    actual_text = actual_text.strip("|")
    actual_text = actual_text.strip()
    headers.append(actual_text)
print(headers)

['Ln', 'Ground state', 'Excited state', 'Final state', 'Emission wavelength (nm)', 'Energy (cm−1)', 'Intensity', 'Remarks']


In [64]:
info = []
for line in doc[23:]:
    actual_text = h.handle(line)
    actual_text = actual_text.strip()
    actual_text = actual_text.strip("|")
    actual_text = actual_text.replace("![\\[thin space\n\\(1/6-em\\)\\]](https://www.rsc.org/images/entities/char_2009.gif)","")
    actual_text = actual_text.replace("![\\[thin\nspace \\(1/6-em\\)\\]](https://www.rsc.org/images/entities/char_2009.gif)","")
    actual_text = actual_text.strip()
    if actual_text != "":
        info.append(actual_text)
#print(info)
#print(len(info))

In [65]:
#Create dictionary (but maybe I should consider using a pandas dataframe?)
lanthanides = {}
ground_state = False
for item in info:
    #Use this and next if block to sort out ground states
    if ground_state:
        lanthanides[current_lanthanide]["Ground State"] = item
        ground_state = False
    #Find those element symbols!
    if item[0].isupper() and item[1].islower() and len(item) == 2:
        current_lanthanide = item
        lanthanides[item] = {"Ground State": '', "Transitions": {}}
        lanthanides[item]["Transitions"] = {headers[2]: [], headers[3]: [], headers[4]: [], headers[5]: '', headers[6]: '', headers[7]: ''}
        ground_state = True
    #Check for state notation
    if ground_state == False and re.match("^[0-9][A-Z][0-9]", item) != None:
    #Then check for arrow to see if it's the excited state, if not, assume last used excited state
        if "→" in item:
            excited_state = item
            lanthanides[current_lanthanide]["Transitions"]["Excited state"].append(excited_state)
        else:
            lanthanides[current_lanthanide]["Transitions"]["Final state"].append(item)
    if ground_state == False and re.match("^[0-9]{3,4}$", item) != None:
        lanthanides[current_lanthanide]["Transitions"]["Emission wavelength (nm)"].append(item)

In [66]:
i = 0

lanthanides = {}
while i < len(info):
    #Check for element symbol
    if info[i][0].isupper() and info[i][1].islower() and len(info[i]) == 2:
        #Grab that element!
        current_lanthanide = info[i]
        lanthanides[current_lanthanide] = {'Transition': [], headers[4]: [], headers[5]: [], headers[6]: [], headers[7]: []}
        i += 1
        
        #Grab that ground state!
        ground_state = info[i]
        i += 1
        
    if "→" in info[i]:
        #Grab that excited state!
        excited_state = info[i]
        i += 1
    
        #Grab that final state!
        final_state = info[i]
        
        transition = excited_state + ' ' + final_state
        lanthanides[current_lanthanide]['Transition'].append(transition)
        lanthanides[current_lanthanide]['Emission wavelength (nm)'].append(info[i+1])
        lanthanides[current_lanthanide]['Energy (cm−1)'].append(info[i+2])
        lanthanides[current_lanthanide]['Intensity'].append(info[i+3])
        lanthanides[current_lanthanide]['Remarks'].append(info[i+4])
        i += 5
            
    else:
        #Grab that final state!
        final_state = info[i]
        
        transition = excited_state + ' ' + final_state
        lanthanides[current_lanthanide]['Transition'].append(transition)
        lanthanides[current_lanthanide]['Emission wavelength (nm)'].append(info[i+1])
        lanthanides[current_lanthanide]['Energy (cm−1)'].append(info[i+2])
        lanthanides[current_lanthanide]['Intensity'].append(info[i+3])
        lanthanides[current_lanthanide]['Remarks'].append(info[i+4])
        i += 5
    df = pd.DataFrame(lanthanides[current_lanthanide])

In [67]:
for key in lanthanides:
    print(key)
    df = pd.DataFrame(lanthanides[key])
    print(df)

Pr
  Transition Emission wavelength (nm) Energy (cm−1) Intensity    Remarks
0  3P2 → 3H4                      440         22727      Weak         QC
1  3P1 → 3H4                      470         21277      Weak         QC
2  3P1 → 1G4                      872         11468    Medium         QC
3  3P0 → 3H4                      480         20833    Strong  UC and QC
4  3P0 → 3H5                      545         18349      Weak  UC and QC
5  3P0 → 3H6                      606         16502    Medium  UC and QC
6  3P0 → 3F2                      640         15625      Weak  UC and QC
7  1D2 → 3F4                     1037          9643    Medium         QC
Nd
       Transition Emission wavelength (nm) Energy (cm−1)       Intensity  \
0   4D3/2 → 4I9/2                      355         28169            Weak   
1   2P3/2 → 4I9/2                      380         26316            Weak   
2  2P3/2 → 4I11/2                      410         24390          Strong   
3  2P3/2 → 4I13/2                

In [73]:
Yb_check = widgets.Checkbox(
    value=False,
    description='Yb',
    disabled=False
)

In [74]:
Yb_check

Checkbox(value=False, description='Yb')

In [76]:
Yb_check.value

True

In [86]:
tab_contents = ['Yb', 'Nd', 'Er', 'Gd', 'etc.']
children = [widgets.SelectMultiple(
        options=['Apples', 'Oranges', 'Pears'],
        value=['Oranges'],
        #rows=10,
        description='Transitions',
        disabled=False
    ) for name in tab_contents]
tab = widgets.Tab()
tab.children = children
for i in range(len(children)):
    tab.set_title(i, tab_contents[i])
tab

In [60]:
print(wavelength_to_rgb(450))

(0, 70, 255)
